In [1]:
import pandas as pd

# Import of Data
# Source: https://www.bmi.gv.at/412/Nationalratswahlen/Nationalratswahl_2019/
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html
df = pd.read_csv("NRW19.csv", sep=';',thousands = '.', decimal = ',')
# Amtliche Gemeindekennzahl (GKZ)

In [2]:
# Data Preparation
df_reduced = df

## Remove all "Wahlkarten"
df_reduced = df_reduced[~df_reduced.GKZ.str.endswith('99')]
## Remove all aggregated results
df_reduced = df_reduced[~df_reduced.GKZ.str.endswith('00')]

## we will only need the percentages and "GKZ", "Gebietsname", "Stimmen"
import numpy
party_names = df_reduced.columns[numpy.append(df_reduced.columns.str.startswith('%')[1:], [False])]
percentages_names = df_reduced.columns[df_reduced.columns.str.startswith('%')]
df_reduced = df_reduced.drop(party_names, axis=1) 
df_reduced.rename(columns=dict(zip(percentages_names, party_names)),inplace=True)
df_reduced = df_reduced.drop(['Wahlbe-rechtigte', 'Ungültige', 'Gültige'], axis=1)

## Aggregate (i.e., sum up) all the percentages of all small parties
big_parties_name = ['ÖVP', 'SPÖ', 'FPÖ', 'NEOS', 'JETZT', 'GRÜNE', 'others']
small_parties_name = ['KPÖ', 'WANDL', 'BZÖ', 'BIER', 'CPÖ', 'GILT', 'SLP']
df_reduced['others'] = df_reduced[small_parties_name].sum(axis=1)
df_reduced = df_reduced.drop(small_parties_name, axis=1)

## Add a column with the respective name of the state
bundeslaender = ['Burgenland', 'Kärnten', 'Niederösterreich', 'Oberösterreich', 'Salzburg', 'Steiermark', 'Tirol', 'Vorarlberg', 'Wien']
bundeslaender[0-1]
def find_state (value):
   if int(value) == 1:
      return bundeslaender[int(value)-1]
   if int(value) == 2:
      return bundeslaender[int(value)-1]
   if int(value) == 3:
      return bundeslaender[int(value)-1]
   if int(value) == 4:
      return bundeslaender[int(value)-1]
   if int(value) == 5:
      return bundeslaender[int(value)-1]
   if int(value) == 6:
      return bundeslaender[int(value)-1]
   if int(value) == 7:
      return bundeslaender[int(value)-1]
   if int(value) == 8:
      return bundeslaender[int(value)-1]
   if int(value) == 9:
      return bundeslaender[int(value)-1]
   return 'Other'

df_reduced['Bundesland'] = df_reduced.GKZ.str[1].apply(find_state)

#pd.set_option('display.max_rows', 3)
df_reduced


,GKZ,Gebietsname,Stimmen,ÖVP,SPÖ,FPÖ,NEOS,JETZT,GRÜNE,others,Bundesland
8,G10101,Eisenstadt,7192,41.69,20.38,14.38,7.52,1.69,13.27,1.07,Burgenland
11,G10201,Rust,1055,34.23,30.38,20.15,5.01,1.16,7.91,1.16,Burgenland
14,G10301,Breitenbrunn am Neusiedler See,1137,33.87,29.82,13.42,6.94,3.42,11.89,0.63,Burgenland
15,G10302,Donnerskirchen,1076,44.38,22.85,18.89,5.67,1.23,6.33,0.66,Burgenland
16,G10303,Großhöflein,1164,42.76,23.79,16.07,5.88,0.61,10.10,0.79,Burgenland
...,...,...,...,...,...,...,...,...,...,...,...
2433,G91901,Döbling,25563,32.26,21.64,10.86,13.34,3.22,17.17,1.50,Wien
2436,G92001,Brigittenau,22535,20.07,36.48,14.62,5.83,2.92,17.83,2.25,Wien
2439,G92101,Floridsdorf,56568,25.99,30.49,20.13,6.65,2.84,11.94,1.95,Wien
2442,G92201,Donaustadt,69291,25.83,29.42,18.26,7.78,2.96,13.79,1.97,Wien


In [3]:
# https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
PCATRNSFORM = pca.fit_transform(df_reduced[big_parties_name])
data = df_reduced[['Gebietsname', 'Stimmen', 'Bundesland']].assign(x = PCATRNSFORM[:,0]).assign(y = PCATRNSFORM[:,1])

#pd.set_option('display.max_rows', 100)

In [4]:
df_reduced

,GKZ,Gebietsname,Stimmen,ÖVP,SPÖ,FPÖ,NEOS,JETZT,GRÜNE,others,Bundesland
8,G10101,Eisenstadt,7192,41.69,20.38,14.38,7.52,1.69,13.27,1.07,Burgenland
11,G10201,Rust,1055,34.23,30.38,20.15,5.01,1.16,7.91,1.16,Burgenland
14,G10301,Breitenbrunn am Neusiedler See,1137,33.87,29.82,13.42,6.94,3.42,11.89,0.63,Burgenland
15,G10302,Donnerskirchen,1076,44.38,22.85,18.89,5.67,1.23,6.33,0.66,Burgenland
16,G10303,Großhöflein,1164,42.76,23.79,16.07,5.88,0.61,10.10,0.79,Burgenland
...,...,...,...,...,...,...,...,...,...,...,...
2433,G91901,Döbling,25563,32.26,21.64,10.86,13.34,3.22,17.17,1.50,Wien
2436,G92001,Brigittenau,22535,20.07,36.48,14.62,5.83,2.92,17.83,2.25,Wien
2439,G92101,Floridsdorf,56568,25.99,30.49,20.13,6.65,2.84,11.94,1.95,Wien
2442,G92201,Donaustadt,69291,25.83,29.42,18.26,7.78,2.96,13.79,1.97,Wien


In [12]:
import numpy as np
import matplotlib.pyplot as plt
from bokeh.plotting import figure, output_notebook, show
from bokeh.transform import factor_cmap
from bokeh.models import ColumnDataSource, HoverTool, Arrow, NormalHead, Label

TOOLTIPS = [("Gemeinde", "@Gebietsname"), ("Bundesland", "@Bundesland"), ("Stimmen", "@Stimmen")]

# Source: https://docs.bokeh.org/en/latest/docs/gallery/color_scatter.html
TOOLS = "hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"
fig = figure(tools=TOOLS, tooltips=TOOLTIPS)

data["radius"] = (df_reduced["Stimmen"] - min(df_reduced["Stimmen"])) / (max(df_reduced["Stimmen"]) - min(df_reduced["Stimmen"])) * (2.5) + 0.3

fig.scatter(x="x", y="y", source=ColumnDataSource(data=data), line_color="black", line_width=0.2, fill_alpha=0.6,
            radius="radius", fill_color=factor_cmap("Bundesland", "Category10_9", list(bundeslaender)))

# Source: http://docs.bokeh.org/en/1.0.2/docs/user_guide/examples/plotting_arrow.html
for i in range(len(big_parties_name)):
    divider = .0325
    fig.add_layout(Arrow(end=NormalHead(size=7), x_end=pca.components_.T[i,0]/divider, y_end=pca.components_.T[i,1]/divider))
    fig.add_layout(Label(x=pca.components_.T[i,0]/divider, y=pca.components_.T[i,1]/divider, text=big_parties_name[i]))
  
output_notebook()
show(fig)

Loading BokehJS ...